---
### 코드 및 리소스 준비
* https://github.com/kakaobrain/pororo 참고하여 pororo 설치
* wav2letter 부분이 설치가 되지 않을 확률이 높음.
* 설치 메뉴얼은  추후 배포 예정

---
### 기본 사용법

In [66]:
import IPython.display as ipd
from pororo import Pororo 
from glob import glob
wavs = glob('sample_data/aihub/*.wav')

In [ ]:
asr = Pororo(task='asr', lang='ko')

In [79]:
wav = random.choice(wavs)
display(ipd.Audio(wav))
out = asr(wav)
out['results']

kwargs: {}


[{'speech_section': '0:00:00 ~ 0:00:06',
  'length_ms': 6060.0,
  'speech': '단 기대대가 와서 그 치료를 하고 자서 쪼끔 마음에 노이는 거 같아'}]

---
### 코드 분석

In [81]:
type(asr)

pororo.tasks.automatic_speech_recognition.PororoASR

In [82]:
# wav 파일 로드 및 전처리
signal = asr._preprocess_audio(wav)
signal.shape

(96939,)

In [83]:
# Speech To Text
model = asr._model
out = model.predict(wav, signal)
out['results'][0]['speech']

'단 기대대가 와서 그 치료를 하고 자서 쪼끔 마음에 노이는 거 같아'

---
### 뉴럴넷 구조 및 입출력

In [89]:
w2v2_net = model.model[0]
w2v2_net

BrainWav2VecCtc(
  (w2v_encoder): BrainWav2VecEncoder(
    (w2v_model): Wav2Vec2Model(
      (feature_extractor): ConvFeatureExtractionModel(
        (conv_layers): ModuleList(
          (0): Sequential(
            (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
            (1): Dropout(p=0.0, inplace=False)
            (2): Fp32GroupNorm(512, 512, eps=1e-05, affine=True)
            (3): GELU()
          )
          (1): Sequential(
            (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
            (1): Dropout(p=0.0, inplace=False)
            (2): GELU()
          )
          (2): Sequential(
            (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
            (1): Dropout(p=0.0, inplace=False)
            (2): GELU()
          )
          (3): Sequential(
            (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
            (1): Dropout(p=0.0, inplace=False)
            (2): GELU()
          )
          (

In [87]:
# 입력 살펴보기
model._last_sample

{'net_input': {'source': tensor([[ 0.0079,  0.0079,  0.0079,  ..., -0.0174, -0.0090, -0.0006]]),
  'padding_mask': tensor([[False, False, False,  ..., False, False, False]])}}

In [78]:
model._last_sample['net_input']['source'].shape

torch.Size([1, 53248])

In [88]:
# 출력 살펴보기
model._last_hypo

[[{'tokens': tensor([15,  6, 10,  5,  8,  9, 15, 23, 15, 23,  8,  6,  5,  7, 36, 19, 12,  5,
            8, 11,  5, 32,  9, 18, 45, 18, 11, 17,  5, 22,  6,  8, 13,  5, 16,  6,
           19, 12,  5, 41, 13, 33, 11, 30,  5, 21,  6,  7, 11, 30,  7, 20,  5, 14,
           13,  7,  9, 14, 11, 10,  5,  8, 12,  5,  8,  6, 46,  7,  6,  5]),
   'score': 0}]]

In [41]:
symbols = model.target_dict.symbols
symbols;

---
### 뉴럴넷 출력값 변환 

In [52]:
idx = model._last_hypo[0][0]['tokens']
ss = [symbols[e] for e in idx]
print(ss)


['ᄒ', 'ᅡ', 'ᄀ', 'ᅵ', 'ᆫ', '|', 'ᄋ', 'ᅭ', 'ᆨ', 'ᄒ', 'ᅡ', 'ᆫ', 'ᄃ', 'ᅡ', 'ᄀ', 'ᅩ', '|', 'ᄀ', 'ᅵ', 'ᄇ', 'ᅮ', 'ᆫ', 'ᄋ', 'ᅵ', '|', 'ᄌ', 'ᅩ', 'ᇂ', 'ᄌ', 'ᅵ', 'ᆫ', '|', 'ᄋ', 'ᅡ', 'ᆭ', 'ᄃ', 'ᅥ', 'ᄅ', 'ᅡ', '|', 'ᄂ', 'ᅥ', 'ᄋ', 'ᅴ', '|', 'ᄆ', 'ᅡ', 'ᆯ', 'ᄃ', 'ᅢ', 'ᄅ', 'ᅩ', '|', 'ᄋ', 'ᅳ', 'ᆷ', 'ᄋ', 'ᅡ', 'ᆨ', 'ᄋ', 'ᅳ', 'ᆯ', '|', 'ᄃ', 'ᅳ', 'ᆯ', 'ᄋ', 'ᅳ', 'ᄆ', 'ᅧ', 'ᆫ', 'ᄉ', 'ᅥ', '|', 'ᄆ', 'ᅡ', 'ᄋ', 'ᅳ', 'ᆷ', 'ᄋ', 'ᅳ', 'ᆯ', '|', 'ᄑ', 'ᅮ', 'ᆯ', 'ᄋ', 'ᅥ', 'ᄋ', 'ᅣ', 'ᄀ', 'ᅦ', 'ᆻ', 'ᄋ', 'ᅥ', '|', 'ᄂ', 'ᅡ', 'ᄂ', 'ᅳ', 'ᆫ', '|', 'ᄌ', 'ᅡ', 'ᆫ', 'ᄌ', 'ᅡ', 'ᆫ', 'ᄒ', 'ᅡ', 'ᆫ', '|', 'ᄋ', 'ᅳ', 'ᆷ', 'ᄋ', 'ᅡ', 'ᆨ', 'ᄋ', 'ᅵ', '|', 'ᄌ', 'ᅩ', 'ᇂ', 'ᄋ', 'ᅡ', '|']


'하긴|욕한다고|기분이|좋진|않더라|너의|말대로|음악을|들으면서|마음을|풀어야겠어|나는|잔잔한|음악이|좋아|'

In [53]:
import unicodedata
ss = ''.join(ss)
txt = unicodedata.normalize('NFC', ss)
txt = txt.replace('|', ' ')
txt

'하긴 욕한다고 기분이 좋진 않더라 너의 말대로 음악을 들으면서 마음을 풀어야겠어 나는 잔잔한 음악이 좋아 '